# Extracting parameters

In [1]:
import numpy as np
import time
import pysqkit
from pysqkit import QubitSystem
from pysqkit.util.linalg import get_mat_elem
from pysqkit.util.phys import temperature_to_thermalenergy
from typing import List, Dict, Callable
import json
import cmath

from IPython.display import display, Latex

The following file contains the parameters used in the simulations. Parameter set 2 and 3 are the relevant ones for the Cross-Resonance gate

In [2]:
with open('flx_transm_params.txt') as param_file:
    parameters_set = json.load(param_file)

In [7]:
temperature = 0.020 # K
thermal_energy = temperature_to_thermalenergy(temperature) # kb T/h in GHz
d_comp = 4

p_set = "3"


#Transmon
levels_t = 3
transm = pysqkit.qubits.SimpleTransmon(
    label='T', 
    max_freq=parameters_set[p_set]["max_freq_t"], 
    anharm=parameters_set[p_set]["anharm_t"],
    diel_loss_tan=parameters_set[p_set]["diel_loss_tan_t"], #set to zero to check d_1 L1 = d_2 L2
    env_thermal_energy=thermal_energy,    
    dim_hilbert=levels_t,
    dephasing_times=parameters_set[p_set]["dephasing_times_t"]
)

#Fluxonium
levels_f = 5

flx = pysqkit.qubits.Fluxonium(
    label='F', 
    charge_energy=parameters_set[p_set]["charge_energy_f"], 
    induct_energy=parameters_set[p_set]["induct_energy_f"], 
    joseph_energy=parameters_set[p_set]["joseph_energy_f"], #8.0, 
    diel_loss_tan=parameters_set[p_set]["diel_loss_tan_f"], #set to zero to check d_1 L1 = d_2 L2
    env_thermal_energy=thermal_energy,
    dephasing_times= parameters_set[p_set]["dephasing_times_f"] #ns/2*np.pi 
)
flx.diagonalize_basis(levels_f)

# We also add a drive on the fluxonium
flx.add_drive(
    pysqkit.drives.microwave_drive,
    label='cz_drive_f',
    pulse=pysqkit.drives.pulses.cos_modulation,
    pulse_shape=pysqkit.drives.pulse_shapes.gaussian_top
)

d_leak = levels_t*levels_f - d_comp

jc = parameters_set[p_set]["jc"]
coupled_sys = transm.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=jc)
bare_system = transm.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=0.0)

freq_flx = flx.state('1')[0] - flx.state('0')[0]
freq_03_flx = flx.state('3')[0] - flx.state('0')[0]
freq_34_flx = flx.state('4')[0] - flx.state('3')[0]

state_label = ["00", "01", "10", "11"]
comp_states = {}
for label in state_label:
    state_tmp = coupled_sys.state(label)[1]
    loc = np.argmax(np.abs(state_tmp))
    phase = cmath.phase(state_tmp[loc])
    state_tmp = np.exp(-1j*phase)*state_tmp
    comp_states[label] = state_tmp

eps_drive = 0.6 #parameter taken in simulations

In [4]:
def mu_yz_flx(
    comp_states: Dict, 
    system: QubitSystem
) -> float:
    
    """
    Description
    --------------------------------------------------------------
    Returns the YZ coefficient associated with the charge 
    operator of the fluxonium in the dressed basis.
    """
    
    op = coupled_sys["F"].charge_op()
    yz0 = get_mat_elem(op, comp_states['00'], comp_states['10'])
    yz1 = get_mat_elem(op, comp_states['01'], comp_states['11'] )
    return (np.imag(yz0 - yz1))/2

def gate_time_cr(
    comp_states: Dict, 
    system: QubitSystem, 
    eps_drive: float
) -> float:
    
    """
    Description
    --------------------------------------------------------------
    Returns the gate time for an ideal cross-resonance gate with
    constant envelope.
    """
    
    coeff = np.abs(mu_yz_flx(comp_states, system))*eps_drive/2
    return 1/(8*coeff)  
    

def zz(
    system: QubitSystem
) -> float:
    
    """
    Description
    --------------------------------------------------------------
    Returns the residual ZZ coupling at zero drive.
    """
    
    xi_zz = system.state('00')[0] + system.state('11')[0] \
        - system.state('01')[0] - system.state('10')[0]
    return xi_zz

def delta(
    system: QubitSystem
) -> float:
    
    """
    Description
    --------------------------------------------------------------
    Returns the parameter Delta that determines the gate speed 
    of the CPHASE gate.
    """
        
    delta_gate = (system.state('13')[0] - system.state('10')[0]) - \
        (system.state('03')[0] - system.state('00')[0])
    return delta_gate 

The following is a list of the relevant parameters

In [6]:
display(Latex(r'$\omega_{{01, flx}}/2 \pi = {:.3f} \, \mathrm{{GHz}}$'.format(freq_flx)))
display(Latex(r'$\omega_{{01, transm}}/2 \pi = {:.3f} \, \mathrm{{GHz}}$'.format(transm.max_freq)))
display(Latex(r'$\omega_{{34, flx}}/2 \pi = {:.3f} \, \mathrm{{GHz}}$'.format(freq_34_flx)))
display(Latex(r'$\omega_{{03, flx}}/2 \pi = {:.3f} \, \mathrm{{GHz}}$'.format(freq_03_flx)))
display(Latex(r'$\xi_{{ZZ}}/h = {:.3f} \, \mathrm{{MHz}}$'.format(1e3*zz(coupled_sys))))
display(Latex(r'$t_{{gate}}^{{(CR)}} = {:.3f} \, \mathrm{{ns}}$'.format(gate_time_cr(comp_states, coupled_sys, eps_drive))))
display(Latex(r'$\Delta/h = {:.3f} \, \mathrm{{MHz}}$'.format(1e3*delta(coupled_sys))))
display(Latex(r'$T_{{1,transm}} = {:.3f} \, \mu s$'.format(1e-3*1/(2*np.pi*transm.loss_rates(0, 1)[0]))))
display(Latex(r'$T_{{1,flx}}^{{1 \mapsto 0}} = {:.3f} \, \mu s$'.format(1e-3*1/(2*np.pi*flx.loss_rates(0, 1)[0]))))
display(Latex(r'$T_{{1,flx}}^{{0 \mapsto 1}} = {:.3f} \, \mu s$'.format(1e-3*1/(2*np.pi*flx.loss_rates(0, 1)[1]))))
display(Latex(r'$T_{{1,flx}}^{{3 \mapsto 0}} = {:.3f} \, \mu s$'.format(1e-3*1/(2*np.pi*flx.loss_rates(0, 3)[0]))))
display(Latex(r'$T_{{1,flx}}^{{0 \mapsto 3}} = {:.3f} \, \mu s$'.format(1e-3*1/(2*np.pi*flx.loss_rates(0, 3)[1]))))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>